<img src="Label.png">

In [56]:
import numpy as np
import pandas as pd

<img src="steps.png">

In [57]:
df= pd.read_csv('SMSSpamCollection.tsv',sep='\t')

In [58]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [59]:
df.describe()

,length,punct
count,5572.000000,5572.000000
mean,80.489950,4.177495
std,59.942907,4.623919
min,2.000000,0.000000
25%,36.000000,2.000000
50%,62.000000,3.000000
75%,122.000000,6.000000
max,910.000000,133.000000


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
 2   length   5572 non-null   int64 
 3   punct    5572 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 174.2+ KB


# Checking Null Values

In [61]:
df.isnull().sum() # no null values

label      0
message    0
length     0
punct      0
dtype: int64

# Counting the Labels

In [62]:
df['label'].value_counts() 

ham     4825
spam     747
Name: label, dtype: int64

# splitting data into test and train set

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
X = df['message']
y = df['label']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer

It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text

In [67]:
count_vect=CountVectorizer()

In [68]:
X[:2]

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
Name: message, dtype: object

# Fit the data to vectorizer
1> Build vocab.
2> Count number of words.

In [69]:
# Fit the vectorizer to the data  (build a vocab, count the number of words)
count_vect.fit(X_train)
X_train_counts=count_vect.transform(X_train)


In [70]:
X_train_counts

<3900x7263 sparse matrix of type '<class 'numpy.int64'>'
	with 52150 stored elements in Compressed Sparse Row format>

In [71]:
X_train.shape

(3900,)

In [72]:
from sklearn.feature_extraction.text import TfidfTransformer

In [73]:
tfidf_transformer=TfidfTransformer()

In [74]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [75]:
X_train_tfidf.shape

(3900, 7263)

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer
The TfidfVectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allow you to encode new documents.

In [77]:
vectorizer=TfidfVectorizer()

In [78]:
X_train_tfidf=vectorizer.fit_transform(X_train)

In [79]:
from sklearn.svm import LinearSVC

# LinearSVC (support vector classifier)

The objective of a Linear SVC (Support Vector Classifier) is to fit to the data you provide, returning a "best fit" hyperplane that divides, or categorizes, your data

In [80]:
clf=LinearSVC()

In [81]:
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [82]:
from sklearn.pipeline import Pipeline

In [83]:
text_clf=Pipeline([('tfidf', TfidfVectorizer()), ('clf',LinearSVC())])

In [84]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [85]:
predictions=text_clf.predict(X_test)

In [86]:
from sklearn.metrics import confusion_matrix, classification_report

In [87]:
print(confusion_matrix(y_test, predictions))

[[1445    3]
 [  10  214]]


In [88]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1448
        spam       0.99      0.96      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.98      0.98      1672
weighted avg       0.99      0.99      0.99      1672



In [89]:
from sklearn import metrics

In [90]:
metrics.accuracy_score(y_test,predictions)

0.9922248803827751

In [91]:
text_clf.predict(["Hello, my Name is Abhishek?"])

array(['ham'], dtype=object)

In [92]:
text_clf.predict(["Free car wash with mobile phone"])

array(['spam'], dtype=object)